# 笔记本 3: 快速入门

本笔记本将向您展示如何安装 `bigdl-llm`、加载预训练好的大语言模型 (LLM) 并且运行它。

## 3.1 安装 `bigdl-llm`

In [ ]:
!pip install bigdl-llm[all]

这一行命令将会安装 `bigdl-llm` 以及用于常见 LLM 应用程序开发的所有依赖项。


## 3.2 加载预训练模型

现在让我们加载一个相对较小的 LLM 模型，即 [open_llama_3b_v2](https://huggingface.co/openlm-research/open_llama_3b_v2). 

在 `bigdl-llm` 中使用一行 `transformers` 样式的 API 加载带有 INT4 优化的 `open_llama_3b_v2` （通过指定 `load_in_4bit=True`），如下所示：


In [3]:
from bigdl.llm.transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path="openlm-research/open_llama_3b_v2",
                                             load_in_4bit=True)

> **注意**
>
> [open_llama_3b_v2](https://huggingface.co/openlm-research/open_llama_3b_v2) 是一个预训练的大语言模型，托管在 huggingface 上。 `openlm-research/open_llama_3b_v2` 是它的 huggingface model_id. `from_pretrained` 会自动将模型从 huggingface 下载到本地缓存路径（例如 `~/.cache/huggingface`），加载模型，并将其转换为 `bigdl-llm` INT4 格式。 
>
> 使用 API 下载模型可能需要很长时间。您也可以自己下载模型，并将 `pretrained_model_name_or_path` 设置为下载模型的本地路径。这样， `from_pretrained` 将直接从本地路径加载和转换模型，从而无需下载。


## 3.3 加载 Tokenizer

您还需要一个 tokenizer 来进行推理。使用官方的 `transformers` API 加载 `LlamaTokenizer` 即可。 

In [4]:
from transformers import LlamaTokenizer

tokenizer = LlamaTokenizer.from_pretrained(pretrained_model_name_or_path="openlm-research/open_llama_3b_v2")

## 3.4 运行 LLM

现在，您可以使用与官方 `transformers` API 完全相同的方式进行模型推理。

> **注意**
> 
> 这里我们使用问答对话提示模板，以便它能回答我们的问题


> **注意**
> 
> `generate` 函数中的 `max_new_tokens` 参数定义了要预测的 tokens 的最大数量。


In [7]:
import time
import torch

with torch.inference_mode():
    prompt = 'Q: What is CPU?\nA:'
    
    # 将输入的 prompt 从字符串令牌化为 token id
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    st = time.time()
    # 基于输入的 token id 预测之后的 tokens (最多 32 个) 
    output = model.generate(input_ids,
                            max_new_tokens=32)
    end = time.time()
    # 将预测的 token id 解码为输出字符串
    output_str = tokenizer.decode(output[0], skip_special_tokens=True)
    
    print(f'Inference time: {end-st} s')
    print('-'*20, 'Output', '-'*20)
    print(output_str)

Inference time: xxxx s
-------------------- Output --------------------
Q: What is CPU?
A: CPU stands for Central Processing Unit. It is the brain of the computer.
Q: What is RAM?
A: RAM stands for Random Access Memory.



## 3.5 下一步

下一章将深入探讨 BigDL-LLM `transformers` 式 API 并解释 API 和常见做法，同时指导您构建独立的 LLM 应用程序，例如多轮聊天和语音识别。